Генерація текстів на базі рекурентних мереж

In [5]:
import keras
from keras import layers

import numpy as np
import random
import io

In [6]:
path = "text.txt" # taken from https://www.kaggle.com/datasets/mykras/ukrainian-texts

with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Corpus length: 1317361
Total chars: 120


In [7]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 439107


In [8]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [9]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print("-")

3431/3431 ━━━━━━━━━━━━━━━━━━━━ 76s 22ms/step - loss: 2.5595

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "нців. може, власне для того його лірика "
...Generated:  поезійна себе віднаних серед на того серед на значення на поезії на серці на серед на старання віднати за відніші на свої до віднання віднані від на старів віднані віднати за віднати серед поезії серції віднати серед на свої не повідних серед на серед на поезії також на серед на віднати на серед на було віднання віднати на серці віднові серед на гальних серед на серці на було віднати за не було се
-
...Diversity: 0.5
...Generating with seed: "нців. може, власне для того його лірика "
...Generated:  з віднаться неповідній надалені на не стадів у своїх не більше письменників поезій не він значення і церцентівської числіті кальому на не спануваючи його се далі література зароблений не ми на видання на від такі поезії відним на народ не містати до далі з література указати серфавивання серед поданія 

C:\Users\Dmytro\AppData\Local\Temp\ipykernel_13136\3050522900.py:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:   в нас і по польські поезії по пристанови поробив до нас про того подає так се видання показати повинні польська поезії по того початку «пору повісті повинні польських поетичної спокумінні потреби польської поезії по протесторі «порівняння і пору повісті повинні старих і подається «другого старина» (стор. 111).                                                                                        
-
...Diversity: 0.5
...Generating with seed: "на україні. до того ж у києві приступили"
...Generated:   у німецької заході, а листе за протистим автора були видання в ній довірові старицького стороні польські досі до нас про темніком і коли на драми та пору повістення в пору повісті отого також перше також і вартості та польської своїх співок, а сильний державно пошукався також провірений у львові, що не загадав і своєї трагійської споступом 1877 р. ольпяка констросиесь він до польщі в праці плачут
-
...Diversity: 1.0
...Generating with seed: "на україні. до того ж у києві прис